# Linear Regression Model - RFM Weights
This notebook aims to use linear regression to find optimal R, F, M weights for Customer Segmentation.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm
from dateutil.relativedelta import relativedelta
from scipy import stats

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Change to your own directory
try:
    os.chdir("/content/drive/MyDrive/BT4211 Data-driven Marketing")
    print("Directory changed")
except OSError:
    print("Error: Can't change the Current Working Directory")

Directory changed


# Import Data

In [7]:
# Load dataset
purchase_df = pd.read_csv("Data/purchase_clean.csv")
purchase_df['Order Date'] = pd.to_datetime(purchase_df['Order Date']) # To convert to Date Format

In [8]:
purchase_df.head()

,Invoice ID,Name,Order Date,State,City,Categories,Subcategories,Ratings,Quantity,MRP,Final Price,GST%,City_Tier,Discount%,Delivery Fee%,Brand,Sale,Covid
0,OD103719706054443200,G3M1R1,2015-08-23,Delhi,New Delhi,Home & Kitchen,Kitchen Appliances,4.1,1,1370.33,1545.0,18%,Tier_1,0,13,Nova,No,No
1,OD106420064045076000,G3M1R1,2016-07-01,Delhi,New Delhi,Health & Personal Care Appliances,Health Care,4.3,1,1164.76,1260.0,5%,Tier_1,0,9,Omron,No,No
2,OD106420064045076001,G3M1R1,2016-07-01,Delhi,New Delhi,Clothing and Accessories,Books,4.5,1,596.67,730.0,5%,Tier_1,0,23,Johnson,No,No
3,OD110363877142631000,G3M1R3,2017-09-30,Delhi,New Delhi,Home Decor,Wall Decor Items,3.5,1,209.00,209.0,12%,Tier_1,0,0,Bexton,No,No
4,OD110510509516810000,G3M1R3,2017-10-17,Delhi,New Delhi,Exercise & Fitness,Yoga,4.7,1,696.00,626.0,28%,Tier_1,11,0,Proline,No,No


In [9]:
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2969 entries, 0 to 2968
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Invoice ID     2969 non-null   object        
 1   Name           2969 non-null   object        
 2   Order Date     2969 non-null   datetime64[ns]
 3   State          2969 non-null   object        
 4   City           2969 non-null   object        
 5   Categories     2969 non-null   object        
 6   Subcategories  2969 non-null   object        
 7   Ratings        2969 non-null   float64       
 8   Quantity       2969 non-null   int64         
 9   MRP            2969 non-null   float64       
 10  Final Price    2969 non-null   float64       
 11  GST%           2969 non-null   object        
 12  City_Tier      2969 non-null   object        
 13  Discount%      2969 non-null   int64         
 14  Delivery Fee%  2969 non-null   int64         
 15  Brand          2953 n

##Linear Regression to get weights for R, F, M (split by month)
Using R, F, M to predict whether a customer will purchase in the latest **month**

In [10]:
# Define invoice month
purchase_df['Invoice Month'] = purchase_df['Order Date'].dt.to_period('M').astype(str)

# Get latest month
latest_month = max(purchase_df['Invoice Month']) #2021-12

In [11]:
# Remove customer ids, which have only records for the last month
# Split into 2 dataframes to check customer ids
purchase_df_latest = purchase_df[purchase_df['Invoice Month'] == latest_month]
purchase_df_others = purchase_df[purchase_df['Invoice Month'] != latest_month]

# Get lists of ids
name_latest = purchase_df_latest['Name'].unique().tolist()
name_others = purchase_df_others['Name'].unique().tolist()

# Define customer ids, who made a purchase during the last month only
new_cust_name = list(set(name_latest) - set(name_others)) # empty list: no new customers in 2021-12

# Remove the customer ids without previous purchasing activity
purchase_df = purchase_df[~purchase_df['Name'].isin(new_cust_name)]

In [ ]:
purchase_df.shape

(2969, 19)

In [12]:
# Features and training set obtained from other months, latest month used for testing
df_X = purchase_df[purchase_df['Invoice Month'] != latest_month]

# Create RFM features
now = dt.datetime(int(latest_month[0:4]), int(latest_month[5:7]), 1)

df_features = df_X.groupby('Name').agg({'Order Date': lambda x: (now.year - x.max().year)*12 + (now.month - x.max().month),
                                        'Invoice ID': pd.Series.nunique,
                                        'Final Price': np.sum}).reset_index()
                                        # 'Quantity': ['mean', 'sum']}).reset_index()

# Rename columns 
df_features.columns = ['Name', 'Recency', 'Frequency', 
                       'Monetary']
                      #  'QuantityAvg', 'QuantityTotal']

df_features

,Name,Recency,Frequency,Monetary
0,G1M1R1,8,2,2187.0
1,G1M1R2,7,7,1768.0
2,G1M1R3,4,5,73573.0
3,G1M1R4,4,3,4293.0
4,G1M1R5,12,5,2450.0
...,...,...,...,...
288,G6M4R9,16,2,2197.0
289,G6M5R1,3,4,1516.0
290,G6M5R2,3,11,64952.0
291,G6M5R3,3,9,2886.0


In [13]:
# Get monthly transactions for customers
cust_month_trans = pd.pivot_table(data=purchase_df,
                                  index=['Name'],
                                  values='Invoice ID',
                                  columns=['Invoice Month'],
                                  aggfunc=pd.Series.nunique,
                                  fill_value=0).reset_index()

cust_month_trans = cust_month_trans.rename_axis('index',axis=1)

cust_month_trans

index,Name,2015-04,2015-08,2015-10,2015-12,2016-01,2016-02,2016-03,2016-07,2016-08,...,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12
0,G1M1R1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,G1M1R2,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,G1M1R3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
3,G1M1R4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,G1M1R5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,G6M4R9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
289,G6M5R1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,2,0,0,0
290,G6M5R2,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,3,4,0,0,0
291,G6M5R3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0


In [14]:
# Define y and X for further model training and prediction
customer_id = ['Name']
target = [latest_month]
y = cust_month_trans[target]
cols = [col for col in df_features if col not in customer_id]
X = df_features[cols]

In [15]:
# Feature Scaling
scaler = StandardScaler()

scaler.fit(X)

X_scaled = pd.DataFrame(scaler.transform(X), columns = X.columns)

In [16]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y[latest_month], test_size=0.25, random_state=123)

# Instantiate linear regression model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
train_pred_y = model.predict(X_train)
test_pred_y = model.predict(X_test)

# Look at the models' coefficients 
model.coef_

array([-3.34220758e-03,  1.21655220e-04,  7.90056657e-07])

In [17]:
# Calculate MSE
mse_train = np.sqrt(mean_squared_error(y_train, train_pred_y))
mse_test = np.sqrt(mean_squared_error(y_test, test_pred_y))

# Print MSE
print('MSE train: {:.3f}; MSE test: {:.3f}'.format(mse_train, mse_test))

# Calculate R-squared and p-values
y_train = np.array(y_train)
ols_reg = sm.OLS(y_train, X_train)
ols_reg = ols_reg.fit()

# Print OLS regression results
print(ols_reg.summary())

MSE train: 0.277; MSE test: 0.131
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.051
Model:                            OLS   Adj. R-squared (uncentered):              0.038
Method:                 Least Squares   F-statistic:                              3.903
Date:                Fri, 24 Jun 2022   Prob (F-statistic):                     0.00961
Time:                        18:17:32   Log-Likelihood:                         -31.359
No. Observations:                 219   AIC:                                      68.72
Df Residuals:                     216   BIC:                                      78.88
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-------

##Linear Regression to get weights for R, F, M (split by quarter)
Using R, F, M to predict whether a customer will purchase in the latest **quarter**

In [18]:
# Define invoice quarter
purchase_df['Invoice Quarter'] = purchase_df['Order Date'].dt.to_period('Q')

# Get latest quarter
latest_quarter = max(purchase_df['Invoice Quarter']) #2021Q4

In [19]:
# Remove customer ids, which have only records for the last quarter
# Split into 2 dataframes to check customer ids
purchase_df_latest = purchase_df[purchase_df['Invoice Quarter'] == latest_quarter]
purchase_df_others = purchase_df[purchase_df['Invoice Quarter'] != latest_quarter]

# Get lists of ids
name_latest = purchase_df_latest['Name'].unique().tolist()
name_others = purchase_df_others['Name'].unique().tolist()

# Define customer ids, who made a purchase during the last quarter only
new_cust_name = list(set(name_latest) - set(name_others)) # 2 new customers

# Remove the customer ids without previous purchasing activity
purchase_df = purchase_df[~purchase_df['Name'].isin(new_cust_name)]

In [20]:
purchase_df.shape

(2967, 20)

In [21]:
# Features and training set obtained from other quarters, latest quarter used for testing
df_X = purchase_df[purchase_df['Invoice Quarter'] != latest_quarter]

# Create RFM features
now = dt.datetime(latest_quarter.year, 3 * ((latest_quarter.month - 1) // 3) + 1, 1)

df_features = df_X.groupby('Name').agg({'Order Date': lambda x: (now.year - x.max().year)*12 + (now.month - x.max().month)//3,
                                        'Invoice ID': pd.Series.nunique,
                                        'Final Price': np.sum}).reset_index()
                                        # 'Quantity': ['mean', 'sum']}).reset_index()

# Rename columns 
df_features.columns = ['Name', 'Recency', 'Frequency', 
                       'Monetary']
                      #  'QuantityAvg', 'QuantityTotal']

df_features

,Name,Recency,Frequency,Monetary
0,G1M1R1,2,2,2187.0
1,G1M1R2,1,7,1768.0
2,G1M1R3,0,5,73573.0
3,G1M1R4,0,3,4293.0
4,G1M1R5,11,5,2450.0
...,...,...,...,...
286,G6M4R9,12,2,2197.0
287,G6M5R1,0,4,1516.0
288,G6M5R2,0,11,64952.0
289,G6M5R3,0,9,2886.0


In [22]:
# Get quarterly transactions for customers
cust_month_trans = pd.pivot_table(data=purchase_df,
                                  index=['Name'],
                                  values='Invoice ID',
                                  columns=['Invoice Quarter'],
                                  aggfunc=pd.Series.nunique,
                                  fill_value=0).reset_index()

cust_month_trans = cust_month_trans.rename_axis('index',axis=1)

cust_month_trans

index,Name,2015Q2,2015Q3,2015Q4,2016Q1,2016Q3,2017Q1,2017Q3,2017Q4,2018Q1,...,2019Q3,2019Q4,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4
0,G1M1R1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1,G1M1R2,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,1,0,0
2,G1M1R3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,2,0
3,G1M1R4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,1,0
4,G1M1R5,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,G6M4R9,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
287,G6M5R1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
288,G6M5R2,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,1,7,0
289,G6M5R3,0,0,0,0,0,0,0,0,0,...,0,1,0,0,2,0,0,1,2,0


In [23]:
# Define y and X for further model training and prediction
customer_id = ['Name']
target = [latest_quarter]
y = cust_month_trans[target]
cols = [col for col in df_features.columns if col not in customer_id]
X = df_features[cols]

In [24]:
# Feature Scaling
scaler = StandardScaler()

scaler.fit(X)

X_scaled = pd.DataFrame(scaler.transform(X), columns = X.columns)

In [25]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y[latest_quarter], test_size=0.25, random_state=123)

# Instantiate linear regression model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
train_pred_y = model.predict(X_train)
test_pred_y = model.predict(X_test)

# Look at the models' coefficients 
model.coef_

array([-6.94958030e-03,  2.86062536e-02, -4.26114821e-06])

In [26]:
# Calculate MSE
mse_train = np.sqrt(mean_squared_error(y_train, train_pred_y))
mse_test = np.sqrt(mean_squared_error(y_test, test_pred_y))

# Print MSE
print('MSE train: {:.3f}; MSE test: {:.3f}'.format(mse_train, mse_test))

# Calculate R-squared and p-values
y_train = np.array(y_train)
ols_reg = sm.OLS(y_train, X_train)
ols_reg = ols_reg.fit()

# Print OLS regression results
print(ols_reg.summary())

MSE train: 0.609; MSE test: 0.632
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.218
Model:                            OLS   Adj. R-squared (uncentered):              0.207
Method:                 Least Squares   F-statistic:                              19.99
Date:                Fri, 24 Jun 2022   Prob (F-statistic):                    1.81e-11
Time:                        18:17:45   Log-Likelihood:                         -202.88
No. Observations:                 218   AIC:                                      411.8
Df Residuals:                     215   BIC:                                      421.9
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-------

##Linear Regression to get weights for R, F, M (split by year)
Using R, F, M to predict whether a customer will purchase in the latest **year**

In [27]:
# Define invoice year
purchase_df['Invoice Year'] = purchase_df['Order Date'].dt.to_period('Y')

# Get latest year
latest_year = max(purchase_df['Invoice Year']) #2021

In [28]:
# Remove customer ids, which have only records for the last year
# Split into 2 dataframes to check customer ids
purchase_df_latest = purchase_df[purchase_df['Invoice Year'] == latest_year]
purchase_df_others = purchase_df[purchase_df['Invoice Year'] != latest_year]

# Get lists of ids
name_latest = purchase_df_latest['Name'].unique().tolist()
name_others = purchase_df_others['Name'].unique().tolist()

# Define customer ids, who made a purchase during the last year only
new_cust_name = list(set(name_latest) - set(name_others))

# Remove the customer ids without previous purchasing activity
purchase_df = purchase_df[~purchase_df['Name'].isin(new_cust_name)]

In [29]:
purchase_df.shape

(2850, 21)

In [30]:
# Features and training set obtained from other months, latest month used for testing
df_X = purchase_df[purchase_df['Invoice Year'] != latest_year]

# Create RFM features
now = dt.datetime(latest_year.year, 1, 1)

df_features = df_X.groupby('Name').agg({'Order Date': lambda x: (now.year - x.max().year),
                                        'Invoice ID': pd.Series.nunique,
                                        'Final Price': np.sum}).reset_index()
                                        # 'Quantity': ['mean', 'sum']}).reset_index()

# Rename columns 
df_features.columns = ['Name', 'Recency', 'Frequency', 
                       'Monetary']
                      #  'QuantityAvg', 'QuantityTotal']

df_features

,Name,Recency,Frequency,Monetary
0,G1M1R2,1,5,1400.0
1,G1M1R3,1,3,9629.0
2,G1M1R4,1,2,1394.0
3,G1M1R5,1,5,2450.0
4,G1M1R6,1,2,2078.0
...,...,...,...,...
254,G6M4R8,1,3,29097.0
255,G6M4R9,1,2,2197.0
256,G6M5R2,1,3,53187.0
257,G6M5R3,1,6,1989.0


In [31]:
# Get monthly transactions for customers
cust_month_trans = pd.pivot_table(data=purchase_df,
                                  index=['Name'],
                                  values='Invoice ID',
                                  columns=['Invoice Year'],
                                  aggfunc=pd.Series.nunique,
                                  fill_value=0).reset_index()

cust_month_trans = cust_month_trans.rename_axis('index',axis=1)

cust_month_trans

index,Name,2015,2016,2017,2018,2019,2020,2021
0,G1M1R2,0,0,0,0,3,2,2
1,G1M1R3,0,0,0,0,1,2,2
2,G1M1R4,0,0,0,0,0,2,1
3,G1M1R5,0,0,0,0,0,5,0
4,G1M1R6,0,0,0,0,0,2,2
...,...,...,...,...,...,...,...,...
254,G6M4R8,0,0,0,0,1,2,0
255,G6M4R9,0,0,0,0,1,1,0
256,G6M5R2,0,0,0,0,0,3,8
257,G6M5R3,0,0,0,3,1,2,3


In [32]:
# Define y and X for further model training and prediction
customer_id = ['Name']
target = [latest_year]
y = cust_month_trans[target]
cols = [col for col in df_features.columns if col not in customer_id]
X = df_features[cols]

# X['FxM'] = X['Frequency'] * np.log(X['Monetary'])
# X = X[['Recency', 'FxM']]

In [33]:
# Feature Scaling
scaler = StandardScaler()

scaler.fit(X)

X_scaled = pd.DataFrame(scaler.transform(X), columns = X.columns)

In [34]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y[latest_year], test_size=0.25, random_state=123)

# Instantiate linear regression model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
train_pred_y = model.predict(X_train)
test_pred_y = model.predict(X_test)

# Look at the models' coefficients 
model.coef_

array([-6.76962147e-01,  1.98895582e-01,  1.15063761e-05])

In [35]:
# Calculate MSE
mse_train = np.sqrt(mean_squared_error(y_train, train_pred_y))
mse_test = np.sqrt(mean_squared_error(y_test, test_pred_y))

# Print MSE
print('MSE train: {:.3f}; MSE test: {:.3f}'.format(mse_train, mse_test))

# Calculate R-squared and p-values
y_train = np.array(y_train)
ols_reg = sm.OLS(y_train, X_train)
ols_reg = ols_reg.fit()

# Print OLS regression results
print(ols_reg.summary())

MSE train: 4.842; MSE test: 4.988
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.387
Model:                            OLS   Adj. R-squared (uncentered):              0.377
Method:                 Least Squares   F-statistic:                              40.13
Date:                Fri, 24 Jun 2022   Prob (F-statistic):                    3.69e-20
Time:                        18:17:56   Log-Likelihood:                         -584.81
No. Observations:                 194   AIC:                                      1176.
Df Residuals:                     191   BIC:                                      1185.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-------